In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/RecFldGrn


# Read Raw Data

In [2]:
import os 
import pandas as pd

RecName = 'PNSect'
folder = 'data/Simulations/'
raw_file_path =  os.path.join(folder, RecName) + '.csv'
print(raw_file_path)


data/Simulations/PNSect.csv


In [3]:
# load all records
df = pd.read_csv(raw_file_path)
df

,PID,ECID,PNID,PNSectID,SectName
0,P0,EC21,72374,140,History of Present Illness
1,P0,EC21,72374,142,Physical Exam
2,P0,EC21,72374,143,Assessment/Plan
3,P0,EC21,72374,144,Patient Instructions
4,P0,EC21,72374,145,Problem List/Past Medical History
...,...,...,...,...,...
277,P7,EC17,435,44,Problem List/Past Medical History
278,P7,EC17,435,43,Assessment/Plan
279,P7,EC17,435,42,Physical Exam
280,P7,EC17,435,41,Review of Systems


# Preprocess

## Adding Missing Records

In [4]:
from recfldgrn.datapoint import load_df_data_from_folder

prefix_cols = ['PID', 'ECID', 'PNID']
focal_cols = ['PNSectID']

Parent_RecName = 'PN'
rec_folder = 'data/ProcData/RecFolder/'
fullrec_folder =  os.path.join(rec_folder, Parent_RecName) 
df_parent = load_df_data_from_folder(fullrec_folder)
df_parent

,PID,ECID,PNID,DT
0,P0,EC18,71800,2023-03-18
1,P0,EC140,72375,NaT
2,P0,EC23,72376,NaT
3,P0,EC141,72377,NaT
4,P0,EC142,72378,NaT
...,...,...,...,...
220,P6,EC15,1335,2023-03-31
221,P6,EC93,72575,NaT
222,P6,EC195,72576,NaT
223,P6,EC173,72577,NaT


In [5]:
import numpy as np

d = pd.merge(df_parent[prefix_cols], df, on = prefix_cols, how = 'left')
for col in focal_cols: 
    d.loc[d[col].isnull(), col] = np.ones(d[col].isnull().sum()).cumsum() + d[col].max()
    d[col] = d[col].astype(int)
    assert len(d[col]) == d[col].nunique()
    
df = d
df

,PID,ECID,PNID,PNSectID,SectName
0,P0,EC18,71800,190,Health Maintenance
1,P0,EC18,71800,188,Social History
2,P0,EC18,71800,178,Chief Complaint
3,P0,EC18,71800,179,History of Present Illness
4,P0,EC18,71800,180,Review of Systems
...,...,...,...,...,...
480,P7,EC17,435,44,Problem List/Past Medical History
481,P7,EC17,435,43,Assessment/Plan
482,P7,EC17,435,42,Physical Exam
483,P7,EC17,435,41,Review of Systems


# Save To Disk by Bucket

In [6]:
from recfldgrn.datapoint import convert_PID_to_PIDgroup
from recfldgrn.datapoint import RANGE_SIZE, write_df_to_folders, load_df_data_from_folder

In [7]:
rec_folder = 'data/ProcData/RecFolder'
fullrec_folder = os.path.join(rec_folder, RecName)
print(df.shape, fullrec_folder)

if not os.path.exists(fullrec_folder): os.makedirs(fullrec_folder)
write_df_to_folders(RecName, fullrec_folder, df)


(485, 5) data/ProcData/RecFolder/PNSect


In [8]:
from recfldgrn.datapoint import PatientDP
print(rec_folder)
PID = 'P4'
Pat = PatientDP(PID, rec_folder, RANGE_SIZE)
print(Pat)
dfx = Pat.get_df_rec(RecName)
dfx

data/ProcData/RecFolder


,PID,ECID,PNID,PNSectID,SectName
0,P4,EC125,72530,437,NaN
1,P4,EC24,72531,438,NaN
2,P4,EC36,72532,439,NaN
3,P4,EC80,72533,440,NaN
4,P4,EC163,72534,441,NaN
5,P4,EC106,72535,442,NaN
6,P4,EC85,72536,443,NaN
7,P4,EC88,72537,444,NaN
8,P4,EC179,72538,445,NaN
9,P4,EC48,72539,446,NaN


In [9]:
from recfldgrn.datapoint import load_df_data_from_folder
print(fullrec_folder)
df = load_df_data_from_folder(fullrec_folder)
df

data/ProcData/RecFolder/PNSect


,PID,ECID,PNID,PNSectID,SectName
0,P0,EC18,71800,190,Health Maintenance
1,P0,EC18,71800,188,Social History
2,P0,EC18,71800,178,Chief Complaint
3,P0,EC18,71800,179,History of Present Illness
4,P0,EC18,71800,180,Review of Systems
...,...,...,...,...,...
480,P7,EC17,435,44,Problem List/Past Medical History
481,P7,EC17,435,43,Assessment/Plan
482,P7,EC17,435,42,Physical Exam
483,P7,EC17,435,41,Review of Systems


# Field & Grain

In [10]:
FieldGrainInfo = []

In [11]:
RecName = 'PNSect'
prefix_ids = ['PID', 'ECID', 'PNID']
focal_ids = ['PNSectID']
field_list = ['SectName']

## Field `SectName`

In [12]:
# (a) field and (b) get field-focused df
field = 'SectName'
dfx = df[prefix_ids + focal_ids + [field]]
dfx

,PID,ECID,PNID,PNSectID,SectName
0,P0,EC18,71800,190,Health Maintenance
1,P0,EC18,71800,188,Social History
2,P0,EC18,71800,178,Chief Complaint
3,P0,EC18,71800,179,History of Present Illness
4,P0,EC18,71800,180,Review of Systems
...,...,...,...,...,...
480,P7,EC17,435,44,Problem List/Past Medical History
481,P7,EC17,435,43,Assessment/Plan
482,P7,EC17,435,42,Physical Exam
483,P7,EC17,435,41,Review of Systems


### Grain `PNSctNmDftGrn`

In [13]:
from recfldgrn.grains import Field2Grain_2_get_GrnStrList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnIdxList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnWgtList_Fn
from recfldgrn.utils import generate_grain_vocab_info

In [14]:
# (c) grain

grain = 'PNSctNmDftGrn'
field2grain = '-'.join([field, grain])
recfield2grain = RecName + '@' + field2grain
print(grain)
print(field2grain)
print(recfield2grain)

PNSctNmDftGrn
SectName-PNSctNmDftGrn
PNSect@SectName-PNSctNmDftGrn


In [15]:
from recfldgrn.grains.misc.dt_grn import load_DTgrn_Dictionary
from transformers import AutoTokenizer

if 'Nume' in grain:
    # print('Current grain is a numeric grain')
    grain_wgt_fn = Field2Grain_2_get_GrnWgtList_Fn[recfield2grain]
    series_wgt = dfx[field].apply(grain_wgt_fn)
    Vocab = {'v2idx': len(series_wgt.iloc[0]) * [1]}
    v2idx = Vocab['v2idx']

elif 'DT' in grain:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = load_DTgrn_Dictionary()
    v2idx = Vocab['v2idx']
    
elif 'Tknz' in grain:
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    v2idx = tokenizer
    Vocab = {'v2idx': v2idx}

else:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = generate_grain_vocab_info(series_str)
    v2idx = Vocab['v2idx']

print(v2idx)
grain_idx_fn = Field2Grain_2_get_GrnIdxList_Fn[recfield2grain]
series_idx = dfx[field].apply(lambda x: grain_idx_fn(x, v2idx))
series_idx


{'_padding': 0, '_missing': 1, 'Procedure': 2, 'Surgical History': 3, 'History of Present Illness': 4, 'Allergies': 5, 'Physical Exam': 6, 'Assessment': 7, 'Plan': 8, 'Medications': 9, 'Problem List': 10, 'Past Medical History': 11, 'Social History': 12, 'Health Maintenance': 13, 'Chief Complaint': 14, 'Family History': 15, 'Review of Systems': 16, 'Patient Instructions': 17, 'Medicare Wellness': 18, 'Immunizations': 19, 'SBIRT': 20, 'Attending Attestation': 21, 'Lab Results': 22}


0          [13]
1          [12]
2          [14]
3           [4]
4          [16]
         ...   
480    [10, 11]
481      [7, 8]
482         [6]
483        [16]
484         [4]
Name: SectName, Length: 485, dtype: object

In [16]:
d = {}
d['RecName'] = RecName

d['prefix_ids'] = prefix_ids
d['focal_ids'] = focal_ids 
d['field_list'] = field_list

d['field'] = field
d['grain'] = grain
d['field2grain'] = field2grain
d['recfield2grain'] = recfield2grain
d['Vocab'] = Vocab

FieldGrainInfo.append(d)

# Save Field Grain Info



In [17]:
df_FieldGrainInfo = pd.DataFrame(FieldGrainInfo)
df_FieldGrainInfo

,RecName,prefix_ids,focal_ids,field_list,field,grain,field2grain,recfield2grain,Vocab
0,PNSect,"[PID, ECID, PNID]",[PNSectID],[SectName],SectName,PNSctNmDftGrn,SectName-PNSctNmDftGrn,PNSect@SectName-PNSctNmDftGrn,"{'idx2v': {0: '_padding', 1: '_missing', 2: 'P..."


In [18]:
fldgrn_folder = 'data/ProcData/FldGrnInfo'

if not os.path.exists(fldgrn_folder): os.makedirs(fldgrn_folder)

fullfldgrn_file = os.path.join(fldgrn_folder, RecName + '.p')
df_FieldGrainInfo.to_pickle(fullfldgrn_file)